In [ ]:
#这一段代码是手动登录获取cookies文件

from selenium import webdriver
import json

# 登录的网址
douyin_url = 'https://www.douyin.com/user/MS4wLjABAAAAXjHAn5JalEMScCl30dVdlVbpMVvoULmPd4c7LBXjWAs?from_tab_name=main'
 
# 启动浏览器
wd = webdriver.Chrome()
 
#浏览器打开网页
wd.get(douyin_url)
 
#等待你输入登录信息
input('输入完成登录后回车继续......')
 
#浏览器登录后获取cookie
cookies = wd.get_cookies()
print(json.dumps(cookies, indent=4, ensure_ascii=False))
 
#将cookies保存在本地
with open("你的文件地址", 'w') as f:
    f.write(json.dumps(cookies, indent=4, ensure_ascii=False))
 
#关闭浏览器
wd.quit()

In [ ]:
#爬取迪士尼所有视频的链接

import json
import time
import csv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

links_collected = None

def getVideoLinks():
    ### 读取 cookies 并自动登陆抖音 ###
    file_path = "你的文件地址"  # 替换为实际的文件路径
    
    with open(file_path, 'r', encoding='utf-8') as f:
        cookies_file = f.read()
    cookie_list = json.loads(cookies_file)
    print("成功读取 cookies 文件。")

    # 初始化 WebDriver
    driver = webdriver.Chrome()
    actions = ActionChains(driver)
    driver.get('https://www.douyin.com/user/MS4wLjABAAAAXjHAn5JalEMScCl30dVdlVbpMVvoULmPd4c7LBXjWAs?from_tab_name=main')
    print("打开抖音主页。")

    # 添加 cookies 并刷新
    for cookie in cookie_list:
        driver.add_cookie(cookie)
    driver.refresh()
    print("成功添加 cookies 并刷新页面。")

    # 确保窗口最大化及等待页面加载完成
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    driver.maximize_window()
    print("窗口最大化并等待页面加载完成。")
    time.sleep(10)  # 等待登录后保存登录的弹窗

    # 开始收集抖音主页所有的视频链接
    print("开始收集视频链接...")
    video_links = set()
    scroll_pause_time = 0.5
    max_scroll_attempts = 5
    scroll_attempts = 0
    last_video_count = 0

    while scroll_attempts < max_scroll_attempts:
        # 模拟键盘 PAGE_DOWN
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(scroll_pause_time)

        # 获取页面源代码
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        for link in soup.find_all(class_="uz1VJwFY TyuBARdT IdxE71f8"):
            href = link['href']
            video_links.add(href)
            print(f"找到视频链接: {href}")
        current_video_count = len(video_links)
        print(f"已收集的视频链接数: {current_video_count}")

        if current_video_count == last_video_count:
            scroll_attempts += 1
            print(f"未加载到新视频，滚动尝试次数: {scroll_attempts}/{max_scroll_attempts}")
        else:
            scroll_attempts = 0
            last_video_count = current_video_count
            print("加载到新视频，继续滚动")
            
    # 最终检查，确保所有视频链接已被捕获
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 修正链接格式
    video_links = {f"http://www.douyin.com{link}" if link.startswith('/video/') else f"http:{link}" if link.startswith('//') else link for link in video_links}
    
    # 保存链接到 CSV 文件
    with open("video_links.csv", "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Links"])  # 写入表头
        for link in video_links:
            writer.writerow([link])
    print("视频链接已保存到文件 video_links.csv 中。")

    print(f"最终收集到的视频链接数: {len(video_links)}")
    driver.quit()  # 关闭浏览器
    return video_links

links_collected = getVideoLinks()

In [ ]:
#爬取评论

import json
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pyautogui

def apply_cookies(driver, cookies_list):
    """将 Cookies 应用于浏览器"""
    if not cookies_list:
        print("Cookies 列表为空，无法应用。")
        return

    # 获取 cookie 的 domain
    cookie_domain = cookies_list[0].get("domain", "").lstrip(".")
    target_url = f"https://{cookie_domain}"  # 构造目标 URL

    # 确保浏览器打开正确的域名
    driver.get(target_url)

    driver.delete_all_cookies()  # 清空已有的 cookies
    for cookie in cookies_list:
        try:
            driver.add_cookie(cookie)
        except Exception as e:
            print(f"添加 cookie 出错: {e}")

    driver.refresh()  # 刷新页面以应用 cookies
    print("Cookies 已成功应用并刷新页面。")


def handle_note_link(driver, link):
    """处理 note 链接"""
    driver.get(link)
    print(f"打开链接: {link}")

    # 等待页面加载完成
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(20)

    comments_data = []
    
    # 尝试找到评论按钮
    try:
        comment_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH,
                                        '//div[contains(@class, "cxpsBymd") and contains(@class, "kNtvycrk") and text()[contains(., "评论")]]'))
        )
        comment_button.click()
        time.sleep(5)  # 等待评论加载
        print("成功点击评论按钮，加载评论。")

        # 按原方式爬取评论
        print("请将鼠标手动放置在评论区上方...")
        time.sleep(3)

        comment_num = driver.find_element(By.CSS_SELECTOR, 'div[class="cxpsBymd aJe8biVj kNtvycrk"]').text
        comment_num = int(re.findall(r'\d+', comment_num)[0]) // 20 + 1

        print("comment_num=", comment_num)
        while comment_num >= 0:
            comment_num -= 1
            username_divs = driver.find_elements(By.CLASS_NAME, 'BT7MlqJC')
            comment_divs = driver.find_elements(By.CLASS_NAME, 'C7LroK_h')
            print(len(username_divs), len(comment_divs))
            for username_div, comment_div in zip(username_divs, comment_divs):
                try:
                    username = username_div.find_elements(By.CLASS_NAME, 'arnSiSbK')
                    username = "".join([user.text.strip() for user in username])
                except:
                    username = "未知用户"
                try:
                    comment_text = comment_div.text.strip()
                except:
                    comment_text = "未找到评论"

                if username or comment_text:
                    if {"用户名": username, "评论": comment_text} not in comments_data:
                        comments_data.append({"用户名": username, "评论": comment_text})
                        print({"用户名": username, "评论": comment_text})

            pyautogui.scroll(-1000)
            time.sleep(0.05)

    except Exception as e:
        print("评论按钮未找到，改用视频爬取方式:", e)

        # 改用视频爬取方式
        last_comment_count = 0
        print("开始滚动加载评论 (备用爬取方式)...")
        while True:
            pyautogui.scroll(-7000)
            time.sleep(1)
            current_comment_count = len(driver.find_elements(By.CLASS_NAME, 'C7LroK_h'))
            if current_comment_count == last_comment_count:
                print("已到达页面底部，无新评论加载。")
                break
            last_comment_count = current_comment_count
            username_divs = driver.find_elements(By.CLASS_NAME, 'BT7MlqJC')
            comment_divs = driver.find_elements(By.CLASS_NAME, 'C7LroK_h')
            for username_div, comment_div in zip(username_divs, comment_divs):
                try:
                    username_elements = username_div.find_elements(By.CLASS_NAME, 'arnSiSbK')
                    username = ''.join([user.text.strip() for user in username_elements])
                except:
                    username = "未知用户"
                try:
                    comment_text = comment_div.text.strip()
                except:
                    comment_text = "未找到评论"
                if username or comment_text:
                    comment_entry = {"用户名": username, "评论": comment_text}
                    if comment_entry not in comments_data:
                        comments_data.append(comment_entry)
                        print(comment_entry)

    print("note 链接处理完成！")
    return comments_data


def handle_video_link(driver, link):
    """处理 video 链接"""
    driver.get(link)
    print(f"打开链接: {link}")

    # 等待页面加载完成
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(5)

    comments_data = []
    last_comment_count = 0

    print("开始滚动加载评论 (video 链接)...")
    while True:
        pyautogui.scroll(-3000)
        time.sleep(1)
        current_comment_count = len(driver.find_elements(By.CLASS_NAME, 'C7LroK_h'))
        if current_comment_count == last_comment_count:
            print("已到达页面底部，无新评论加载。")
            break
        last_comment_count = current_comment_count
        username_divs = driver.find_elements(By.CLASS_NAME, 'BT7MlqJC')
        comment_divs = driver.find_elements(By.CLASS_NAME, 'C7LroK_h')
        for username_div, comment_div in zip(username_divs, comment_divs):
            try:
                username_elements = username_div.find_elements(By.CLASS_NAME, 'arnSiSbK')
                username = ''.join([user.text.strip() for user in username_elements])
            except:
                username = "未知用户"
            try:
                comment_text = comment_div.text.strip()
            except:
                comment_text = "未找到评论"
            if username or comment_text:
                comment_entry = {"用户名": username, "评论": comment_text}
                if comment_entry not in comments_data:
                    comments_data.append(comment_entry)
                    print(comment_entry)

    print("video 链接处理完成！")
    return comments_data

def process_links():
    # 读取 Cookies 文件
    cookies_file = "multiple_cookies.json"  # 替换为你的 Cookies 文件路径
    with open(cookies_file, 'r', encoding='utf-8') as f:
        all_cookies = json.load(f)

    # 提取所有用户的 keys
    user_keys = list(all_cookies.keys())
    if not user_keys:
        print("Cookies 文件为空或未找到用户。")
        return

    # 启动浏览器
    driver = webdriver.Chrome()

    try:
        # 读取链接 CSV 文件
        input_csv = "last_links.csv"  # 替换为实际链接文件
        output_csv = "disney_51-80_lastt.csv"  # 输出文件
        with open(input_csv, 'r', encoding='utf-8') as infile:
            reader = csv.DictReader(infile)
            links = [row["Links"] for row in reader]

        # 打开输出文件
        with open(output_csv, 'w', encoding='utf_8_sig', newline='', errors='ignore') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=["ID", "comments"])
            writer.writeheader()

            # 初始化当前 Cookies 用户索引
            current_cookie_index = 0

            # 遍历链接
            for idx, link in enumerate(links):
                comments_data_return = []
                attempts = 0  # 尝试次数

                while attempts < len(user_keys):
                    # 获取当前用户的 Cookies
                    user_key = user_keys[current_cookie_index]
                    cookies_list = all_cookies.get(user_key, [])
                    if not cookies_list:
                        print(f"未找到用户 {user_key} 的 Cookies，跳过。")
                        current_cookie_index = (current_cookie_index + 1) % len(user_keys)
                        attempts += 1
                        continue

                    print(f"使用用户 {user_key} 的 Cookies 尝试爬取链接: {link}")
                    apply_cookies(driver, cookies_list)  # 应用 Cookies

                    print(f"=========处理第 {idx + 1} 个链接: {link} =========")
                    time.sleep(15)  # 增加延迟避免反爬

                    # 根据链接类型处理
                    if "note" in link:
                        comments_data_return = handle_note_link(driver, link)
                    elif "video" in link:
                        comments_data_return = handle_video_link(driver, link)

                    # 成功爬取评论
                    if comments_data_return:
                        for comment in comments_data_return:
                            writer.writerow({"ID": comment["用户名"], "comments": comment["评论"]})
                        print(f"评论爬取成功，切换到下一个 Cookies 用户处理新链接...")
                        # 切换到下一个 Cookies
                        current_cookie_index = (current_cookie_index + 1) % len(user_keys)
                        break  # 跳出尝试循环，进入下一个链接

                    # 未成功爬取评论
                    print(f"Cookies 用户 {user_key} 未能爬取到评论，切换到下一个 Cookies...")
                    current_cookie_index = (current_cookie_index + 1) % len(user_keys)
                    attempts += 1

                # 如果尝试了所有 Cookies 仍未爬取成功
                if not comments_data_return:
                    print(f"所有 Cookies 都无法处理链接: {link}，跳过该链接。")

        print(f"所有链接处理完成，结果已保存到 {output_csv}")
    finally:
        driver.quit()



# 执行主程序
process_links()